In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report, precision_recall_fscore_support
import warnings
import os 
from scipy.stats import f_oneway
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
df1 = pd.read_excel('case_study1.xlsx')
df2 = pd.read_excel('case_study2.xlsx')

In [3]:
df1.head()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,...,0,0,1,0,4,1,4,0,72,18
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,...,0,1,0,0,0,0,1,0,7,7
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,...,0,6,1,0,0,2,6,0,47,2
3,4,1,0,1,1,0,1.000,0.0,1.000,0.000,...,0,0,0,0,0,0,1,1,5,5
4,5,3,2,1,0,0,0.000,0.0,0.333,0.667,...,0,0,0,0,0,3,0,2,131,32


In [4]:
df2.head()

,PROSPECTID,time_since_recent_payment,time_since_first_deliquency,time_since_recent_deliquency,num_times_delinquent,max_delinquency_level,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,549,35,15,11,29,29,0,0,0,...,0.0,0.0,0.0,13.333,1,0,PL,PL,696,P2
1,2,47,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,0.860,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,302,11,3,9,25,25,1,9,8,...,0.0,0.0,0.0,5741.667,1,0,ConsumerLoan,others,693,P2
3,4,-99999,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,9.900,0,0,others,others,673,P2
4,5,583,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,-99999.000,0,0,AL,AL,753,P1


In [5]:
# remove null values

df1 = df1.loc[df1['Age_Oldest_TL']!=-99999]

In [6]:
columns_to_be_removed=[]

for i in df2.columns:
    if df2.loc[df2[i]==-99999].shape[0]>10000:
        columns_to_be_removed.append(i)

In [7]:
df2=df2.drop(columns=columns_to_be_removed,axis=1)

In [8]:
# removing rest of the null values along with the row

for i in df2.columns:
    df2 = df2.loc[df2[i]!=-99999]

In [9]:
# checking common columns name
for i in list(df1.columns):
    if i in list(df2.columns):
        print(i)

PROSPECTID


In [10]:
df=pd.merge(df1,df2,how='inner',left_on=['PROSPECTID'],right_on=['PROSPECTID'])

In [11]:
df.head()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,...,0.0,0.0,0.000,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,...,0.0,0.0,0.000,0.0,1,0,ConsumerLoan,others,693,P2
3,5,3,2,1,0,0,0.000,0.0,0.333,0.667,...,0.0,0.0,0.000,0.0,0,0,AL,AL,753,P1
4,6,6,5,1,0,0,0.000,0.0,0.167,0.833,...,1.0,0.0,0.429,0.0,1,0,ConsumerLoan,PL,668,P3


In [12]:
# checking for categorical variables
for i in df.columns:
    if df[i].dtype == 'object':
        print(i)

MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


In [13]:
# chi-squared test b/w cat vs cat 
for i in ['MARITALSTATUS','EDUCATION','GENDER','last_prod_enq2','first_prod_enq2']:
    chi2,pval,_,_=chi2_contingency(pd.crosstab(df[i],df['Approved_Flag']))
    print(i,'-----',pval)

MARITALSTATUS ----- 3.578180861038862e-233
EDUCATION ----- 2.6942265249737532e-30
GENDER ----- 1.907936100186563e-05
last_prod_enq2 ----- 0.0
first_prod_enq2 ----- 7.84997610555419e-287


Since all the categorical features have pval<=0.05 so are failed to reject null hypothesis so we will accept all

In [14]:
# finding all numerical columns
numeric_columns=[]
for i in df.columns:
    if df[i].dtypes!='object' and i not in ['PROSPECTID','Approved_Flag']:
        numeric_columns.append(i)

In [15]:
print(numeric_columns)

['Total_TL', 'Tot_Closed_TL', 'Tot_Active_TL', 'Total_TL_opened_L6M', 'Tot_TL_closed_L6M', 'pct_tl_open_L6M', 'pct_tl_closed_L6M', 'pct_active_tl', 'pct_closed_tl', 'Total_TL_opened_L12M', 'Tot_TL_closed_L12M', 'pct_tl_open_L12M', 'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'Auto_TL', 'CC_TL', 'Consumer_TL', 'Gold_TL', 'Home_TL', 'PL_TL', 'Secured_TL', 'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL', 'Age_Newest_TL', 'time_since_recent_payment', 'num_times_delinquent', 'max_recent_level_of_deliq', 'num_deliq_6mts', 'num_deliq_12mts', 'num_deliq_6_12mts', 'num_times_30p_dpd', 'num_times_60p_dpd', 'num_std', 'num_std_6mts', 'num_std_12mts', 'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt', 'num_dbt_6mts', 'num_dbt_12mts', 'num_lss', 'num_lss_6mts', 'num_lss_12mts', 'recent_level_of_deliq', 'tot_enq', 'CC_enq', 'CC_enq_L6m', 'CC_enq_L12m', 'PL_enq', 'PL_enq_L6m', 'PL_enq_L12m', 'time_since_recent_enq', 'enq_L12m', 'enq_L6m', 'enq_L3m', 'AGE', 'NETMONTHLYINCOME', 'Time_With_Curr_Empr', 'pct_

In [16]:
# before checking the association between numerical columns and target columns, we will check multi-collinearity

vif_data=df[numeric_columns]
total_columns=vif_data.shape[1]
columns_to_be_kept=[]
column_index=0

for i in range(0,total_columns):

    vif_value=variance_inflation_factor(vif_data,column_index)
    print(column_index,'-----',vif_value)

    if vif_value<=6:
        columns_to_be_kept.append(numeric_columns[i])
        column_index+=1
    else:
        vif_data = vif_data.drop([numeric_columns[i]],axis=1)

c:\Users\chirag\anaconda3\envs\crm\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 ----- inf


c:\Users\chirag\anaconda3\envs\crm\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 ----- inf
0 ----- 11.320180023967996
0 ----- 8.363698035000336
0 ----- 6.520647877790928
0 ----- 5.149501618212625
1 ----- 2.611111040579735


c:\Users\chirag\anaconda3\envs\crm\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2 ----- inf
2 ----- 1788.7926256209232
2 ----- 8.601028256477228
2 ----- 3.8328007921530785
3 ----- 6.099653381646739
3 ----- 5.5813520096427585
4 ----- 1.985584353098778


c:\Users\chirag\anaconda3\envs\crm\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


5 ----- inf
5 ----- 4.809538302819343
6 ----- 23.270628983464636
6 ----- 30.595522588100053
6 ----- 4.384346405965583
7 ----- 3.0646584155234247
8 ----- 2.898639771299253
9 ----- 4.377876915347322
10 ----- 2.207853583695844
11 ----- 4.916914200506864
12 ----- 5.214702030064725
13 ----- 3.3861625024231476
14 ----- 7.840583309478997
14 ----- 5.255034641721434


c:\Users\chirag\anaconda3\envs\crm\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


15 ----- inf
15 ----- 7.380634506427232
15 ----- 1.4210050015175733
16 ----- 8.083255010190316
16 ----- 1.6241227524040114
17 ----- 7.257811920140003
17 ----- 15.59624383268298
17 ----- 1.825857047132431
18 ----- 1.5080839450032666
19 ----- 2.172088834824577
20 ----- 2.6233975535272283
21 ----- 2.2959970812106167
22 ----- 7.360578319196439
22 ----- 2.1602387773102554
23 ----- 2.8686288267891467
24 ----- 6.458218003637272
24 ----- 2.8474118865638256
25 ----- 4.753198156284083
26 ----- 16.22735475594825
26 ----- 6.424377256363877
26 ----- 8.887080381808687
26 ----- 2.3804746142952653
27 ----- 8.609513476514548
27 ----- 13.06755093547673
27 ----- 3.500040056654654
28 ----- 1.908795587481377
29 ----- 17.006562234161628
29 ----- 10.730485153719197
29 ----- 2.3538497522950275
30 ----- 22.104855915136433
30 ----- 2.7971639638512915
31 ----- 3.424171203217696
32 ----- 10.175021454450935
32 ----- 6.408710354561301
32 ----- 1.0011511962625614
33 ----- 3.069197305397274
34 ----- 2.809126160064371

In [17]:
# Annova Test to drop numerical columns
columns_to_be_kept_numerical = []

for i in columns_to_be_kept:
    a = list(df[i])
    b = list(df['Approved_Flag'])

    group_p1 = [value for value,group in zip(a,b) if group=='P1']
    group_p2 = [value for value,group in zip(a,b) if group=='P2']
    group_p3 = [value for value,group in zip(a,b) if group=='P3']
    group_p4 = [value for value,group in zip(a,b) if group=='P4']

    f_statistics, p_value = f_oneway(group_p1,group_p2,group_p3,group_p4)

    if p_value<=0.05:
        columns_to_be_kept_numerical.append(i)

In [18]:
len(columns_to_be_kept_numerical)

37

In [19]:
# Label Encoding for categorical feature
df["MARITALSTATUS"].unique()
# df['EDUCATION'].unique()
# df['GENDER'].unique()
# df['last_prod_enq2'].unique()
# df['first_prod_enq2'].unique()

array(['Married', 'Single'], dtype=object)

# Ordinal Feature - Education
SSC - 1
12th - 2
Graduate - 3
Under Graduate - 3
POST-Gradute - 4
Others - 1 (Uneducatedd or documents misplaced)
professional - 3

In [20]:
features = columns_to_be_kept_numerical + ['MARITALSTATUS','EDUCATION','GENDER','last_prod_enq2','first_prod_enq2']
df=df[features+['Approved_Flag']]

In [21]:
df.head()

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,...,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,MARITALSTATUS,EDUCATION,GENDER,last_prod_enq2,first_prod_enq2,Approved_Flag
0,0.000,0.0,0,0.000,0,0,0,4,1,4,...,0.000,0.0,1,0,Married,12TH,M,PL,PL,P2
1,0.000,0.0,0,0.000,0,0,0,0,0,1,...,0.000,0.0,0,0,Single,GRADUATE,F,ConsumerLoan,ConsumerLoan,P2
2,0.125,0.0,0,0.000,1,0,0,0,2,6,...,0.000,0.0,1,0,Married,SSC,M,ConsumerLoan,others,P2
3,0.000,0.0,0,0.000,0,0,0,0,3,0,...,0.000,0.0,0,0,Married,POST-GRADUATE,M,AL,AL,P1
4,0.000,0.0,1,0.167,0,0,0,0,6,0,...,0.429,0.0,1,0,Married,12TH,M,ConsumerLoan,PL,P3


In [22]:
df.loc[df['EDUCATION']=='SSC',['EDUCATION']]=1
df.loc[df['EDUCATION']=='12TH',['EDUCATION']]=2
df.loc[df['EDUCATION']=='GRADUATE',['EDUCATION']]=3
df.loc[df['EDUCATION']=='UNDER GRADUATE',['EDUCATION']]=3
df.loc[df['EDUCATION']=='POST-GRADUATE',['EDUCATION']]=4
df.loc[df['EDUCATION']=='OTHERS',['EDUCATION']]=1
df.loc[df['EDUCATION']=='PROFESSIONAL',['EDUCATION']]=3

In [23]:
df['EDUCATION'].value_counts()

EDUCATION
3    18931
2    11703
1     9532
4     1898
Name: count, dtype: int64

In [24]:
df['EDUCATION'] = df['EDUCATION'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            42064 non-null  float64
 1   pct_tl_closed_L6M          42064 non-null  float64
 2   Tot_TL_closed_L12M         42064 non-null  int64  
 3   pct_tl_closed_L12M         42064 non-null  float64
 4   Tot_Missed_Pmnt            42064 non-null  int64  
 5   CC_TL                      42064 non-null  int64  
 6   Home_TL                    42064 non-null  int64  
 7   PL_TL                      42064 non-null  int64  
 8   Secured_TL                 42064 non-null  int64  
 9   Unsecured_TL               42064 non-null  int64  
 10  Other_TL                   42064 non-null  int64  
 11  Age_Oldest_TL              42064 non-null  int64  
 12  Age_Newest_TL              42064 non-null  int64  
 13  time_since_recent_payment  42064 non-null  int

In [25]:
# one hot encoding
df_encoded = pd.get_dummies(df, columns=['MARITALSTATUS','GENDER','last_prod_enq2','first_prod_enq2'],dtype='int')

In [26]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               42064 non-null  float64
 1   pct_tl_closed_L6M             42064 non-null  float64
 2   Tot_TL_closed_L12M            42064 non-null  int64  
 3   pct_tl_closed_L12M            42064 non-null  float64
 4   Tot_Missed_Pmnt               42064 non-null  int64  
 5   CC_TL                         42064 non-null  int64  
 6   Home_TL                       42064 non-null  int64  
 7   PL_TL                         42064 non-null  int64  
 8   Secured_TL                    42064 non-null  int64  
 9   Unsecured_TL                  42064 non-null  int64  
 10  Other_TL                      42064 non-null  int64  
 11  Age_Oldest_TL                 42064 non-null  int64  
 12  Age_Newest_TL                 42064 non-null  int64  
 13  t

In [27]:
df_encoded.head()

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,...,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,first_prod_enq2_AL,first_prod_enq2_CC,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others
0,0.000,0.0,0,0.000,0,0,0,4,1,4,...,0,0,1,0,0,0,0,0,1,0
1,0.000,0.0,0,0.000,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
2,0.125,0.0,0,0.000,1,0,0,0,2,6,...,1,0,0,0,0,0,0,0,0,1
3,0.000,0.0,0,0.000,0,0,0,0,3,0,...,0,0,0,0,1,0,0,0,0,0
4,0.000,0.0,1,0.167,0,0,0,0,6,0,...,1,0,0,0,0,0,0,0,1,0


In [28]:
for i in df_encoded.columns:
    if df_encoded[i].dtypes=='object':
        print(i)

Approved_Flag


In [29]:
# Random Forest 
y = df_encoded['Approved_Flag']
x = df_encoded.drop(['Approved_Flag'],axis=1)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42)
rf_classifier.fit(x_train,y_train)
y_pred = rf_classifier.predict(x_test)

accuracy = accuracy_score(y_test,y_pred)
print()
print(f'Accuracay Score : {accuracy}')
print()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test,y_pred)

for i,v in enumerate(['p1','p2','p3','p4']):
    print(f"Class {v}:")
    print(f"Precision : {precision[i]}")
    print(f"Recall : {recall[i]}")
    print(f"f1score: {f1_score[i]}")
    print()


Accuracay Score : 0.7636990372043266

Class p1:
Precision : 0.8370457209847597
Recall : 0.7041420118343196
f1score: 0.7648634172469202

Class p2:
Precision : 0.7957519116397621
Recall : 0.9282457879088206
f1score: 0.856907593778591

Class p3:
Precision : 0.4423380726698262
Recall : 0.21132075471698114
f1score: 0.28600612870275793

Class p4:
Precision : 0.7178502879078695
Recall : 0.7269193391642371
f1score: 0.7223563495895703



In [30]:
# XGBoost

xgb_classifier = xgb.XGBClassifier(objective='multi:softmax', num_class=4)

y = df_encoded['Approved_Flag']
x = df_encoded.drop(['Approved_Flag'], axis=1)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

x_train,x_test,y_train,y_test = train_test_split(x,y_encoded,test_size=0.2, random_state=42)

xgb_classifier.fit(x_train,y_train)
y_pred = xgb_classifier.predict(x_test)

accuracy = accuracy_score(y_test,y_pred)
print()
print(f'Accuracay Score : {accuracy:.2f}')
print()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test,y_pred)

for i,v in enumerate(['p1','p2','p3','p4']):
    print(f"Class {v}:")
    print(f"Precision : {precision[i]}")
    print(f"Recall : {recall[i]}")
    print(f"f1score: {f1_score[i]}")
    print()


Accuracay Score : 0.78

Class p1:
Precision : 0.823906083244397
Recall : 0.7613412228796844
f1score: 0.7913890312660175

Class p2:
Precision : 0.8255418233924413
Recall : 0.913577799801784
f1score: 0.8673315769665035

Class p3:
Precision : 0.4756380510440835
Recall : 0.30943396226415093
f1score: 0.37494284407864653

Class p4:
Precision : 0.7342386032977691
Recall : 0.7356656948493683
f1score: 0.7349514563106796



In [31]:
# Decision Tree
# Random Forest 
y = df_encoded['Approved_Flag']
x = df_encoded.drop(['Approved_Flag'],axis=1)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=42)

dt_model = DecisionTreeClassifier(max_depth=20, min_samples_split=10)
dt_model.fit(x_train,y_train)
y_pred = dt_model.predict(x_test)

accuracy = accuracy_score(y_test,y_pred)
print()
print(f'Accuracay Score : {accuracy}')
print()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test,y_pred)

for i,v in enumerate(['p1','p2','p3','p4']):
    print(f"Class {v}:")
    print(f"Precision : {precision[i]}")
    print(f"Recall : {recall[i]}")
    print(f"f1score: {f1_score[i]}")
    print()


Accuracay Score : 0.7104481160109355

Class p1:
Precision : 0.7232580961727183
Recall : 0.7268244575936884
f1score: 0.7250368912936547

Class p2:
Precision : 0.8094035360404119
Recall : 0.8257680872150645
f1score: 0.8175039246467818

Class p3:
Precision : 0.34099920697858843
Recall : 0.32452830188679244
f1score: 0.3325599381283836

Class p4:
Precision : 0.6531440162271805
Recall : 0.6258503401360545
f1score: 0.6392059553349876



### XGBoost works better So we will perform Hypertuning for this

In [32]:
df_encoded['Approved_Flag'].value_counts()

Approved_Flag
P2    25452
P3     6440
P4     5264
P1     4908
Name: count, dtype: int64

In [35]:
# Hyperparamter  tuning
x_train,x_test,y_train,y_test = train_test_split(x,y_encoded,test_size=0.2, random_state=42)

xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=4)

param_grid = {
    'n_estimators':[50,100,200],
    'max_depth':[3,5,7],
    'learning_rate':[0.01,0.1,0.2]
}

grid_search = GridSearchCV(estimator=xgb_model,param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)

grid_search.fit(x_train,y_train)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_class=4,
                                     num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [36]:
# best hyperparameters
print("Best Hyperparamters : ", grid_search.best_params_)

Best Hyperparamters :  {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}


In [38]:
# evaluate the model with the best hyperparamters on the test set

best_model = grid_search.best_params_
accuracy = grid_search.best_score_
print("Test Accuracy: ", accuracy)

Test Accuracy:  0.780184838489198
